## Confiabilidad de la red eléctrica Caracara
En este libro de notas se analizarán los eventos de fallas que se han presentado en el sistema eléctrico que le suministra energía a las áreas de producción del Bloque Caracara.
Los datos de los eventos de fallas son los suministrados en un archivo Excel denominado **EventosRedesCaracara_V3.xlsx**, el cual está alojado en la carpeta **xl**. 

El archivo original entregado por CEPSA es un libro Excel que se denomina "Cuadro control eventos Redes Distribución Bloque Caracara_V3.xlsx". Este libro contiene tres hojas de cálculo: "Calendario", "Grafica Transitorios" y "Base datos". 

Inicialmente estamos interesados en analizar la hoja "Base datos". Esta hoja contiene 2229 registros de eventos y 18 columnas.

### 1. Lee la hoja de cálculo "Base datos" que contiene los eventos de falla de la red
Inicialmente se cargan los paquetes requeridos y luego se lee la hoja de cálculo.

In [1]:
import pandas as pd
import datetime as dt
from time import strftime
from time import gmtime
import time
from datetime import timedelta
from dateutil.parser import parse
import ciso8601

In [137]:
hojaBaseDatos = None
with pd. ExcelFile("xl\EventosRedesCaracara_V3.xlsx") as reader:
    hojaBaseDatos = pd. read_excel(reader, sheet_name='Base datos')


In [138]:
#Corrije la fila 2004, en la cual el año debería ser 2021 y no 2011.
hojaBaseDatos.loc[2004, "Fecha"] = pd.to_datetime('2021-11-11 00:00:00')
hojaBaseDatos.loc[2004, "Año"] = 2021

### 2. Organiza el DataFrame hojaBaseDatos
Se dejará una columna fechaHoraInicial y otra fechaHoraFinal.

In [139]:
hojaBaseDatos = hojaBaseDatos.rename(columns={'hora día': 'horaDia', 
'TIPO DE EVENTO': 'tipoEvento', 'Día': 'dia', 'Mes': 'mes', 'Año': 'anio', 
'Fecha': 'fecha', 'Hora evento': 'horaEvento', 
'Hora normalización': 'horaNormalizacion', 'Duración (HH:MM)': 'duracionHM',
'Duración\n(Horas)': 'duracionHoras', 'RED': 'tensionRed', 'Línea': 'tramoLinea', 
'Equipos afectados': 'equiposAfectados', 'Causas': 'causas', 
'FlagDisparos': 'esDisparo', 'FlagTransit': 'esTransitorio', 'Día2': 'dia2', 
'Column1': 'columna1'})
#hojaBaseDatos.columns

In [140]:
#Elimina los NA en 'horaEvento'.
hojaBaseDatos = hojaBaseDatos.dropna(subset=['horaEvento'])

In [141]:
#Selecciona columnas de interés.
hojaBaseDatos = hojaBaseDatos[['tipoEvento', 'fecha', 'horaEvento',
       'horaNormalizacion', 'duracionHM', 'duracionHoras', 'tensionRed',
       'tramoLinea', 'equiposAfectados', 'causas', 'esDisparo',
       'esTransitorio']]

In [142]:
#Corrije NAs en 4 registros.
hojaBaseDatos.loc[[423, 825, 1109, 1181], "horaNormalizacion"] = \
    hojaBaseDatos.loc[[423, 825, 1109, 1181], "horaEvento"]
hojaBaseDatos.loc[[423, 825, 1109, 1181], "tensionRed"] = '115 kV'
hojaBaseDatos.loc[[423, 825, 1109, 1181], "duracionHM"] = 0
hojaBaseDatos.loc[[423, 825, 1109, 1181], "duracionHoras"] = 0

In [143]:
#Chequea y corrije registros 'equiposAfectados' con NA.
hojaBaseDatos.loc[[3, 4], 'equiposAfectados'] = \
    hojaBaseDatos.loc[2, 'equiposAfectados']

hojaBaseDatos.loc[3, ['horaNormalizacion', 'duracionHM', 'duracionHoras']] = \
    hojaBaseDatos.loc[2, ['horaNormalizacion', 'duracionHM', 'duracionHoras']]

hojaBaseDatos.loc[4, 'horaNormalizacion'] = hojaBaseDatos.loc[4, 'horaEvento']

hojaBaseDatos.loc[395, ['tensionRed', 'tramoLinea', 'equiposAfectados']] = \
    hojaBaseDatos.loc[398, ['tensionRed', 'tramoLinea', 'equiposAfectados']]



In [144]:
#Chequea y corrije registros que tienen 'tramoLinea' con NA.
hojaBaseDatos[hojaBaseDatos['tramoLinea'].isna()]
hojaBaseDatos.loc[576, 'tramoLinea'] = hojaBaseDatos.loc[574, 'tramoLinea']

hojaBaseDatos.loc[[574, 576], 'horaNormalizacion'] = \
    hojaBaseDatos.loc[[574, 576], 'horaEvento']

hojaBaseDatos.loc[[574, 576], ['duracionHM', 'duracionHoras']] = 0

hojaBaseDatos.loc[623:628, 'horaNormalizacion'] = \
    hojaBaseDatos.loc[623:628, 'horaEvento']

hojaBaseDatos.loc[623:628, ['duracionHM', 'duracionHoras']] = 0
hojaBaseDatos.loc[626:628, 'tramoLinea'] = hojaBaseDatos.loc[638, 'tramoLinea'] 
hojaBaseDatos.loc[623:625, 'tramoLinea'] = hojaBaseDatos.loc[622, 'tramoLinea'] 
#hojaBaseDatos.loc[623:628, :]

In [145]:
#Chequea y corrije registros que tienen 'horaNormalizacion' con NA.
horaNormalizacionNa = hojaBaseDatos['horaNormalizacion'].isna()
#horaNormalizacionNa
hojaBaseDatos.loc[horaNormalizacionNa, 'horaNormalizacion'] = \
    hojaBaseDatos.loc[horaNormalizacionNa, 'horaEvento']

hojaBaseDatos.loc[horaNormalizacionNa, ['duracionHM', 'duracionHoras']] = 0

In [146]:
#Chequea y corrije 'duracionHoras' con NA.
#hojaBaseDatos.loc[hojaBaseDatos['duracionHoras'].isna(), 'duracionHM']
hojaBaseDatos.loc[[756, 758, 763, 770, 773, 777, 783, 789, 799], 'duracionHoras'] = 0
#hojaBaseDatos.loc[[756, 758, 763, 770, 773, 777, 783, 789, 799], 'duracionHoras']

In [147]:
hojaBaseDatos.loc[[4, 146, 395, 994], 'horaNormalizacion'].astype(str)
#df["new_col"]= df['A'].astype(str)
#hojaBaseDatos.loc[994, 'horaNormalizacion']

4      16:34:00
146    16:00:00
395    15:10:00
994    19:13:00
Name: horaNormalizacion, dtype: object

In [148]:
#hojaBaseDatos.loc[[4, 146, 395, 994], 'duracionHoras'] = \
hojaBaseDatos.loc[[4, 146, 395, 994], 'duracionHoras'] = (pd.to_datetime(hojaBaseDatos.loc[[4, 146, 395, 994], 'fecha'].dt.strftime('%Y-%m-%d') + ' ' + \
    hojaBaseDatos.loc[[4, 146, 395, 994],'horaNormalizacion'].astype(str)) - \
        pd.to_datetime(hojaBaseDatos.loc[[4, 146, 395, 994], 'fecha'].dt.strftime('%Y-%m-%d') + ' ' + \
            hojaBaseDatos.loc[[4, 146, 395, 994],'horaEvento'].astype(str))).dt.total_seconds()/3600 #Calcula las horas.

In [149]:
hojaBaseDatos.loc[[4, 146, 395, 994], 'duracionHM'] = \
    pd.to_datetime(hojaBaseDatos.loc[[4, 146, 395, 994], \
        'duracionHoras'], unit='h').dt.strftime('%H:%M')

In [150]:
hojaBaseDatos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2220 entries, 0 to 2228
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   tipoEvento         2220 non-null   object        
 1   fecha              2220 non-null   datetime64[ns]
 2   horaEvento         2220 non-null   object        
 3   horaNormalizacion  2220 non-null   object        
 4   duracionHM         2220 non-null   object        
 5   duracionHoras      2220 non-null   object        
 6   tensionRed         2220 non-null   object        
 7   tramoLinea         2220 non-null   object        
 8   equiposAfectados   2220 non-null   object        
 9   causas             2220 non-null   object        
 10  esDisparo          2220 non-null   int64         
 11  esTransitorio      2220 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 290.0+ KB


In [153]:
hojaBaseDatos["horaEvento"] = hojaBaseDatos["horaEvento"].astype(str)
hojaBaseDatos["horaNormalizacion"] = hojaBaseDatos["horaNormalizacion"].astype(str)
hojaBaseDatos["duracionHM"] = hojaBaseDatos["duracionHM"].astype(str)
hojaBaseDatos.loc[1426, "duracionHoras"] = 0
hojaBaseDatos["duracionHoras"] = pd.to_numeric(hojaBaseDatos["duracionHoras"])


In [155]:
#Salva el hojaBaseDatos en formato parquet (más comprimido).
hojaBaseDatos.to_parquet("parquet/eventosRed1.parquet.gzip", compression='gzip') 

In [2]:
#Lee el archivo eventosRed en formato parquet.
eventosRed1 = pd.read_parquet("parquet/eventosRed1.parquet.gzip") 

In [21]:
#Crea columna 'fechaHoraInicial'.
eventosRed1['fechaHoraInicial'] = \
    pd.to_datetime(eventosRed1['fecha'].astype(str) + " " + \
        eventosRed1['horaEvento'])

In [227]:
#Corrije 'eventosRed1.loc[1837, 'horaEvento']'
eventosRed1.loc[1837, 'horaNormalizacion'] = eventosRed1.loc[1837, 'horaEvento']
#Corrije 'eventosRed1.iloc[159, :]'
eventosRed1.loc[159,'horaEvento'] = '22:59:00'
eventosRed1.loc[159,'horaNormalizacion'] = '00:28:00'
#eventosRed1.loc[309, 'horaNormalización'] = eventosRed1.loc[309, 'horaNormalización'][-8:]
eventosRed1.loc[744, 'horaEvento'] = '18:08:00'
eventosRed1.loc[744, 'horaNormalizacion'] = '00:14:00'


In [279]:
#vectorLen8 = (eventosRed1['horaEvento'].apply(len) != 8)
#eventosRed1.loc[vectorLen8, 'horaEvento']
#vectorLen8 = (eventosRed1['horaNormalizacion'].apply(len) != 8)
#eventosRed1.loc[vectorLen8, 'horaNormalizacion']
#
for fila in [745, 1096, 1282, 1298, 1301, 1302, 1511]:
    eventosRed1.loc[fila, 'horaEvento'] = eventosRed1.loc[fila, 'horaEvento'][-8:]
#
for fila in [745, 1096, 1282, 1298, 1301, 1302, 1511]:
    eventosRed1.loc[fila, 'horaNormalizacion'] = eventosRed1.loc[fila, 'horaNormalizacion'][-8:]

In [352]:
#Crea columna 'fechaHoraInicial'.
eventosRed1['fechaHoraInicial'] = \
    pd.to_datetime(eventosRed1['fecha'].astype(str) + " " + \
        eventosRed1['horaEvento'])
#Crea columna 'fechaHoraFinal'.
eventosRed1['fechaHoraFinal'] = \
    pd.to_datetime(eventosRed1['fecha'].astype(str) + " " + \
        eventosRed1['horaNormalizacion'])

C:\Users\ericd\AppData\Local\Temp\ipykernel_8024\2327504831.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventosRed1['fechaHoraInicial'] = \
C:\Users\ericd\AppData\Local\Temp\ipykernel_8024\2327504831.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventosRed1['fechaHoraFinal'] = \


In [312]:
eventosRed1['fechaHoraFinalAjus'] = eventosRed1['fechaHoraInicial'] + \
    (pd.to_timedelta(round(eventosRed1['duracionHoras']*3600, 0), 's'))

In [362]:
fechasDiferentes = (eventosRed1['fechaHoraFinal'] != eventosRed1['fechaHoraFinalAjus'])
#fechasDiferentes
eventosRed1.loc[fechasDiferentes, ['fecha', 'horaEvento', 'horaNormalizacion', \
    'duracionHM', 'duracionHoras', 'fechaHoraFinal', 'fechaHoraFinalAjus']]

,fecha,horaEvento,horaNormalizacion,duracionHM,duracionHoras,fechaHoraFinal,fechaHoraFinalAjus


In [354]:
eventosRed1.loc[[159, 309, 744, 745, 1096, 1282, 1298], 'fechaHoraFinal'] = \
    eventosRed1.loc[[159, 309, 744, 745, 1096, 1282, 1298], 'fechaHoraFinalAjus']

In [355]:
eventosRed1.loc[[1269, 1275, 1277, 1299, 1300, 1315, 1321, 1323, 1359, 1366, \
    1382, 1403, 1404, 1407, 1420, 1421, 1433, 1472], 'horaNormalizacion'] = \
        eventosRed1.loc[[1269, 1275, 1277, 1299, 1300, 1315, 1321, 1323, 1359, \
            1366, 1382, 1403, 1404, 1407, 1420, 1421, 1433, 1472], \
                'horaEvento']
#
eventosRed1.loc[[1269, 1275, 1277, 1299, 1300, 1315, 1321, 1323, 1359, 1366, \
    1382, 1403, 1404, 1407, 1420, 1421, 1433, 1472], 'fechaHoraFinal'] = \
        eventosRed1.loc[[1269, 1275, 1277, 1299, 1300, 1315, 1321, 1323, 1359, \
            1366, 1382, 1403, 1404, 1407, 1420, 1421, 1433, 1472], \
                'fechaHoraFinalAjus']

In [356]:
eventosRed1.loc[1107, 'duracionHM'] = '00:06:00'
eventosRed1.loc[1107, 'duracionHoras'] = 6/60

In [357]:
eventosRed1.loc[[1415, 1419, 1515, 1680, 1770, 1892], 'fechaHoraFinal'] = \
        eventosRed1.loc[[1415, 1419, 1515, 1680, 1770, 1892], \
                'fechaHoraFinalAjus']

In [359]:
eventosRed1.loc[[1107, 2059, 2060], 'fechaHoraFinalAjus'] = \
        eventosRed1.loc[[1107, 2059, 2060], 'fechaHoraFinal']

In [361]:
#eventosRed1.loc[[1384, 1511], 'fechaHoraFinal'] = \
#        eventosRed1.loc[[1384, 1511], 'fechaHoraFinalAjus']
eventosRed1.loc[1384, 'horaNormalizacion'] = \
        eventosRed1.loc[1384, 'horaEvento']

In [393]:
duplicadosFechaHoraInicial = eventosRed1[eventosRed1.duplicated(subset=['fechaHoraInicial'], keep=False)]
#eventosRed1[duplicadosFechaHoraInicial]
#duplicadosFechaHoraInicial[['fechaHoraInicial', 'duracionHM', 'tensionRed', 'tramoLinea', 'equiposAfectados', 'causas']]

### 3. Corrije los valores de la columna 'tensionRed'
Las tres tensiones son '34,5 kV', '115 kV' y '230 kV'.

In [4]:
#pd.unique(eventosRed1['tensionRed'])
#array(['115 kV', '34,5 kV', '230 kV', '115 KV', '34,5 KV', '230KV',
#       '115KV', '34,5KV'], dtype=object)
#Corrije '230 KV'.
eventosRed1.loc[eventosRed1['tensionRed'] == '230 KV', 'tensionRed'] = '230 kV'
#Corrije '230KV'.
eventosRed1.loc[eventosRed1['tensionRed'] == '230KV', 'tensionRed'] = '230 kV'
#Corrije '115 KV'.
eventosRed1.loc[eventosRed1['tensionRed'] == '115 KV', 'tensionRed'] = '115 kV'
#Corrije '115KV'.
eventosRed1.loc[eventosRed1['tensionRed'] == '115KV', 'tensionRed'] = '115 kV'
#Corrije '34,5 KV'.
eventosRed1.loc[eventosRed1['tensionRed'] == '34,5 KV', 'tensionRed'] = '34,5 kV'
#Corrije '34,5KV'.
eventosRed1.loc[eventosRed1['tensionRed'] == '34,5KV', 'tensionRed'] = '34,5 kV'


In [ ]:
# Resetea el index, para tener el índice con los números secuenciales. Ok.
#eventosRed1 = eventosRed1.reset_index()

### 4. Corrije los valores de la columna 'tramoLinea'
Son muchos los tramos, sobre todo los de 115 kV, que son 1810.

#### 4.1. Clasifica los tramos de línea de 115 kV

In [60]:
#pd.unique(eventosRed1['cadenaTramoLinea'])
#Crea nueva columna que convierte la lista de la columna 'tramoLinea' en str. 
#eventosRed1['cadenaTramoLinea'] = eventosRed1['tramoLinea'].apply(lambda x: ''.join(map(str, x)))
#eventosRed1['tramoLinea'].nunique() #Hay 192 tipos de tramos  de línea.
#Explora '115...'.
#eventosRed1.loc[(eventosRed1['tramoLinea'].str.slice(0,21) == \
#    '115 kV Reforma - Ocoa'), 'tramoLinea']

#eventosRed1.loc[(eventosRed1['tramoLinea'].str.slice(0,17) == '115 kV La Reforma'), \
#    'tramoLinea'].nunique() #Hay 33 tipos de tramos de línea de '115...'.
#tramosLineaUnicosOrdenados = \
#pd.unique(eventosRed1.loc[(eventosRed1['cadenaTramoLinea'].str.slice(0,3) == \
#    '115'), 'cadenaTramoLinea'])
#*******************************************************************************
#tramosLineaUnicosOrdenados:
#*******************************************************************************
# 1. '115 kV Reforma - Ocoa'
#*******************************************************************************
# '115 kV Reforma - Ocoa', 
# '115 kV La Reforma', 
#*******************************************************************************
# 2. '115 kV Ocoa - Suria'
#*******************************************************************************
# '115 kV Ocoa - Suria', 
# '115 kV Suria - Puerto Lopez', 
# '115 KV línea ocoa', 
# '115 KV línea Ocoa', 
# '115 kV Interruptor Acople subestacion Oca']       
#*******************************************************************************
# 3. '115 kV Ocoa - Santa Helena'
#*******************************************************************************
# '115 KV , Ocoa - S/E Santa Helena', 
# '115 kV Ocoa - Santa Helena', 
# '115 kv Ocoa- Santa-Helena',
# '115 KV Ocoa-Santa Helena', 
#*******************************************************************************
# 4. '115 kV Suria - Puerto López'
#*******************************************************************************
# '115 KV Suria-Puerto Lopez', 
# '115 kV  Puerto lopez - Campo Bonito', 
# '115 kV Suria-Ecopetrol', 
#*******************************************************************************
# 5. '115 kV Puerto López - Campo Bonito'
#*******************************************************************************
# '115 kV Puerto López - Campo Bonito',
# '115 kV Puerto Lopez - Campo Bonito',
# '115 kV Puerto Lopez - Puerto Campo Bonito', 
# '115 kV  Puerto López - Campo Bonito.', 
# '115 kV Puerto Lopez - Puerto Gaitán', 
#*******************************************************************************
# 6. '115 kV Campo Bonito - Puerto Gaitán'
#******************************************************************************* 
# '115 kV Campo Bonito -Puerto Gaitán',
# '115 kV Campo Bonito- Puerto Gaitán', 
# '115 KV Puerto Gaitan- Cristalina ', 
# '115 kV  Campo Bonito- Puerto Gaitan,\n ',
# '115 kV Campo Bonito - Puerto Gaitán', 
# '115 kV  Reforma - Ocoa',
# '115 kV  Campo Bonito- Puerto Gaitan', 
#*******************************************************************************
# 7. '115 kV Puerto Gaitán - La Cristalina'
#******************************************************************************* 
# '115 KV  Puerto Gaitán -La Cristalina ', 
# '115 KV deslastre de carga la cristalina', 
# '115 KV subestación Gaitán', 
# '115 kV Puerto Gaitán-La Cristalina', 
# '115 kV Puerto Gaitán - La Cristalina', 
# '115 kV   Puerto Gaitan-La Cristalina', 
#******************************************************************************* 
# 8. '115 kV San Fernando', 
#******************************************************************************* 
#******************************************************************************* 
#eventosRed1['tramoAjus'] = False #Crea columna para chequear tramo ajustado.
#******************************************************************************* 
# 1.1. Corrije '115 kV Reforma - Ocoa' y chequea 'tramoAjus'. Ok.
#******************************************************************************* 
#eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(['115 kV Reforma - Ocoa', \
#    '115 kV  Reforma - Ocoa', '115 kV La Reforma', '115 KV línea ocoa', \
#        '115 KV línea Ocoa']), 'tramoLinea'] = '115 kV Reforma - Ocoa'
#eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(['115 kV Reforma - Ocoa', \
#    '115 kV  Reforma - Ocoa', '115 kV La Reforma', '115 KV línea ocoa', \
#        '115 KV línea Ocoa']), 'tramoAjus'] = True
#******************************************************************************* 
# 2.1. Corrije '115 kV Ocoa - Suria' y chequea 'tramoAjus'. Ok.
#******************************************************************************* 
#eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(['115 kV Ocoa - Suria', \
#    '115 kV Interruptor Acople subestacion Oca', '115 kV Suria-Ecopetrol']), \
#        'tramoLinea'] = '115 kV Ocoa - Suria'
#eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(['115 kV Ocoa - Suria', \
#    '115 kV Interruptor Acople subestacion Oca', '115 kV Suria-Ecopetrol']), \
#        'tramoAjus'] = True
#******************************************************************************* 
# 3.1. Corrije '115 kV Ocoa - Santa Helena' y chequea 'tramoAjus'. Ok.
#******************************************************************************* 
#eventosRed1.loc[eventosRed1['cadenaTramoLinea'].\
#    isin(['115 kV Ocoa - Santa Helena', '115 kv Ocoa- Santa-Helena', \
#        '115 KV , Ocoa - S/E Santa Helena', '115 KV Ocoa-Santa Helena']), \
#        'tramoLinea'] = '115 kV Ocoa - Santa Helena'
#eventosRed1.loc[eventosRed1['cadenaTramoLinea'].\
#    isin(['115 kV Ocoa - Santa Helena', '115 kv Ocoa- Santa-Helena', \
#        '115 KV , Ocoa - S/E Santa Helena', '115 KV Ocoa-Santa Helena']), \
#        'tramoAjus'] = True
#******************************************************************************* 
# 4.1. Corrije '115 kV Suria - Puerto López' y chequea 'tramoAjus'. Ok.
#******************************************************************************* 
#eventosRed1.loc[eventosRed1['cadenaTramoLinea'].\
#    isin(['115 kV Suria - Puerto Lopez', '115 KV Suria-Puerto Lopez']), \
#        'tramoLinea'] = '115 kV Suria - Puerto López'
#eventosRed1.loc[eventosRed1['cadenaTramoLinea'].\
#    isin(['115 kV Suria - Puerto Lopez', '115 KV Suria-Puerto Lopez']), \
#        'tramoAjus'] = True
#******************************************************************************* 
# 5.1. Corrije '115 kV Puerto López - Campo Bonito' y chequea 'tramoAjus'. Ok.
#******************************************************************************* 
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].\
#     isin(['115 kV Puerto López - Campo Bonito', \
#         '115 kV Puerto Lopez - Campo Bonito', \
#             '115 kV  Puerto lopez - Campo Bonito', \
#                 '115 kV Puerto Lopez - Puerto Gaitán', \
#                     '115 kV Puerto Lopez - Puerto Campo Bonito', \
#                         '115 kV  Puerto López - Campo Bonito.']), \
#         'tramoLinea'] = '115 kV Puerto López - Campo Bonito'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].\
#     isin(['115 kV Puerto López - Campo Bonito', \
#         '115 kV Puerto Lopez - Campo Bonito', \
#             '115 kV  Puerto lopez - Campo Bonito', \
#                 '115 kV Puerto Lopez - Puerto Gaitán', \
#                     '115 kV Puerto Lopez - Puerto Campo Bonito', \
#                         '115 kV  Puerto López - Campo Bonito.']), \
#         'tramoAjus'] = True
#******************************************************************************* 
# 6.1. Corrije '115 kV Campo Bonito - Puerto Gaitán' y chequea 'tramoAjus'.
#******************************************************************************* 
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].\
#     isin(['115 kV Campo Bonito -Puerto Gaitán', \
#         '115 kV  Campo Bonito- Puerto Gaitan,\n ', \
#             '115 kV Campo Bonito - Puerto Gaitán', \
#                 '115 kV Campo Bonito- Puerto Gaitán', \
#                     '115 kV  Campo Bonito- Puerto Gaitan']), \
#         'tramoLinea'] = '115 kV Campo Bonito - Puerto Gaitán'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].\
#     isin(['115 kV Campo Bonito -Puerto Gaitán', \
#         '115 kV  Campo Bonito- Puerto Gaitan,\n ', \
#             '115 kV Campo Bonito - Puerto Gaitán', \
#                 '115 kV Campo Bonito- Puerto Gaitán', \
#                     '115 kV  Campo Bonito- Puerto Gaitan']), \
#         'tramoAjus'] = True
#******************************************************************************* 
# 6.2. Corrije '115 kV Subestación Puerto Gaitán' y chequea 'tramoAjus'. Ok.
#******************************************************************************* 
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].\
#     isin(['115 KV subestación Gaitán']), \
#         'tramoLinea'] = '115 kV Subestación Puerto Gaitán'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].\
#     isin(['115 KV subestación Gaitán']), \
#         'tramoAjus'] = True
#******************************************************************************* 
# 7.1. Corrije '115 kV Puerto Gaitán - La Cristalina' y chequea 'tramoAjus'. Ok.
#******************************************************************************* 
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].\
#     isin(['115 kV Puerto Gaitán - La Cristalina', \
#         '115 KV  Puerto Gaitán -La Cristalina ', \
#             '115 KV Puerto Gaitan- Cristalina ', \
#                 '115 kV   Puerto Gaitan-La Cristalina', \
#                     '115 KV deslastre de carga la cristalina', \
#                         '115 kV Puerto Gaitán-La Cristalina']), \
#         'tramoLinea'] = '115 kV Puerto Gaitán - La Cristalina'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].\
#     isin(['115 kV Puerto Gaitán - La Cristalina', \
#         '115 KV  Puerto Gaitán -La Cristalina ', \
#             '115 KV Puerto Gaitan- Cristalina ', \
#                 '115 kV   Puerto Gaitan-La Cristalina', \
#                     '115 KV deslastre de carga la cristalina', \
#                         '115 kV Puerto Gaitán-La Cristalina']), \
#         'tramoAjus'] = True
#******************************************************************************* 
# 8.1. Corrije '115 kV San Fernando' y chequea 'tramoAjus'. Ok.
#******************************************************************************* 
#eventosRed1.loc[eventosRed1['cadenaTramoLinea'] == '115 kV San Fernando', \
#    'tramoLinea'] = '115 kV San Fernando'
#eventosRed1.loc[eventosRed1['cadenaTramoLinea'] == '115 kV San Fernando', \
#    'tramoAjus'] = True
#******************************************************************************* 

#### 4.2. Clasifica los tramos de línea de 34,5 kV

In [59]:
#pd.unique(eventosRed1['cadenaTramoLinea'])
#Explora '34,5 kV...'.OK.
#pd.unique(eventosRed1.loc[(eventosRed1['cadenaTramoLinea'].str.slice(0,7) == \
#    '34,5 kV'), 'tramoLinea'])
#Explora '34,5 KV...'.
#pd.unique(eventosRed1.loc[(eventosRed1['cadenaTramoLinea'].str.slice(0,7) == \
#    '34,5 KV'), 'tramoLinea'])
#*******************************************************************************
# 4.2.1. Ajusta a '34_5kV_CCS' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV MCDC 2-2 CCS E - CCS A13', '34,5 kV RC 04 CDC Caracara Sur', \
#     '34,5 kV RC 04 CDC Caracara sur', '34,5 kV RC 04 CDC CCS', \
#         '34,5 kV RC 04 CDC CCS', '34,5 kV Troncal  CCS', \
#             '34,5 kV Troncal Caracara sur', '34,5 kV Troncal CCS']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.2.2. Ajusta a '34_5kV_CCS_CG' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV Centro de generación CCS', '34,5 kV TR 211']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_CCS_CG'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.3. Ajusta a '34_5kV_CCS_R1' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV Ramal 1 CCS', '34,5 kV Ramal 1 CDC-CCS', \
#     '34,5 kV Ramal 1', '34,5 kV Ramal 1-3 CCS C6', '34,5 kV Ramal 1-CDC-CCS']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_CCS_R1'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.4. Ajusta a '34_5kV_CCS_R1' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV RC 24 CCS CCS C6- C19', '34,5 kV Subramal']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_CCS_R1'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.5. Ajusta a '34_5kV_CCS_R2' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV Ramal 2 CCS, S/E CCS A13@A22', \
#     '34,5 kV S/E CCS A10-A14-A18', \
#         '34,5 kV Subramal 2,1 Caracara sur A1-A4-A31']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_CCS_R2'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.6. Ajusta a '34_5kV_CCS_R3' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV S/E BGL 3-8']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_CCS_R3'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.7. Ajusta a '34_5kV_JGR' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV CDC Jaguar', '34,5 kV RC 04 CDC JAGUAR', \
#     '34,5 kV RC 04 CDC Jaguar', '34,5 kV RC 04 CDC JGR', \
#         '34,5 kV RC 07 CDC JGR', '34,5 kV RC 1A CDC Jaguar', \
#             '34,5 kV Troncal Jaguar', '34,5 kV Troncal JGR']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_JGR'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.8. Ajusta a '34_5kV_JGR_CG' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV Centro de generación JGR', \
#     '34,5 kV Troncal Jaguar Centro de Generacion']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_JGR_CG'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.9. Ajusta a '34_5kV_JGR_R1' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV Ramal 1 CDC-JGR', '34,5 kV Ramal 1 Jaguar', \
#     '34,5 kV Ramal 1 JGR', '34,5 kV Ramal 1-CDC-JGR', \
#         '34,5 kV RC 03 RAMAL 1 CDC JAGUAR', '34,5 kV RC 3 Ramal 1 Jaguar', \
#             '34,5 kV Ramal 1-1,6  CDC-JGR']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_JGR_R1'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.10. Ajusta a '34_5kV_JGR_R2' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV Ramal 2 CDC_JGR', '34,5 kV Ramal 2 CDC-JGR', \
#     '34,5 kV Ramal 2 Jaguar', '34,5 kV Ramal 2-CDC-JGR', '34,5 kV Ramal BENG']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_JGR_R2'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.11. Ajusta a '34_5kV_JGR_R2' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV Subramal 2,2 JGR']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_JGR_R2'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.12. Ajusta a '34_5kV_JGR_R2' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV Ramal 3 Jaguar, RC-30', '34,5 kV Ramal 3']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_JGR_R2'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.13. Ajusta a '34_5kV_JSW' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV Ramal JSW', '34,5 kV Troncal JSW']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_JSW'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.14. Ajusta a '34_5kV_SE_PG' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV Baja tension Trafo TP1', \
#     '34,5 kV Banco de Condensadores', '34,5 kV interruptor BT TP1', \
#         '34,5 kV Troncal Jaguar, Caracara', '34,5 kV Troncal JSW - CCS']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_SE_PG'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.15. Ajusta a '34_5kV_CCS' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 KV Cepcolsa 1 Trooncal CCS', '34,5 KV Cepcolsa 1', \
#     '34,5 KV RC1A CDC CCS', '34,5 KV Troncal CCS ', '34,5 KV troncal CCS']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_CCS'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.16. Ajusta a '34_5kV_CCS_R1' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 KV Ramal 1   RC-24  CDC-CCS', \
#     '34,5 KV reconectador CCS Ramal 1', '34,5 KV reconectador 13 (CCM)']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_CCS_R1'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.17. Ajusta a '34_5kV_CCS_R2' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 KV RAMAL 2', '34,5 KV Ramal 2 CDC-CCS,  estación CCS']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_CCS_R2'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.18. Ajusta a '34_5kV_CCS_R3' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 KV Troncal Bengalas\n']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_CCS_R3'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.19. Ajusta a '34_5kV_EMSA_Rural' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 KV San pedro de de Arimena']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_EMSA_Rural'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.20. Ajusta a '34_5kV_JGR' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 KV Cepcolsa 2', '34,5 KV RC 4 CDC Jaguar', \
#     '34,5 KV reconectador JGR RC04', '34,5 KV Troncal Jaguar ', \
#         '34,5 KV troncal JGR']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_JGR'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.21. Ajusta a '34_5kV_JGR_R1' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 KV T7 MCDC-5 hacia el pozo JGR T7.']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_JGR_R1'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.22. Ajusta a '34_5kV_JGR_R3' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 KV reconectador JGR 42']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_JGR_R3'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True
#*******************************************************************************
# 4.2.23. Ajusta a '34_5kV_JSW' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 KV JSW', \
#     '34,5 KV RECONECTADOR CDC 1 (actúan protecciones, pendiente establecer la causa del disparo).']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),\
#     'tramoLinea'] = '34_5kV_JSW'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), \
#     'tramoAjus'] = True

#### 4.3. Clasifica los tramos de línea de 230 kV

In [63]:
#*******************************************************************************
# 4.3.1. Ajusta a '230 kV Reforma' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['230 kV Reforma', '230 kV  Guavio - Reforma', \
#     '230 kV  Reforma-Tunal', '230KV Tunal - Reforma']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '230 kV Reforma'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.3.2. Ajusta a '115 kV Reforma - Ocoa' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['230KV Reforma - Ocoa']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '115 kV Reforma - Ocoa'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True

#### 4.4. Corrije tramo de línea de 114 kV

In [65]:
#*******************************************************************************
# 4.4.1. Ajusta '115 kV Campo Bonito - Puerto Gaitán' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['114 kV  Campo Bonito- Puerto Gaitan']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '115 kV Campo Bonito - Puerto Gaitán'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True

#### 4.5. Corrije tramos que empiezan con 'Apertura' y 'Bengala...'

In [85]:
#*******************************************************************************
# 4.5.1. Ajusta a '34_5kV_CCS_R2' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Apertura reconectador CCSB_006.']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R2'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.5.2. Ajusta a '34_5kV_EMSA_Rural' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
#eventosRed1.loc[eventosRed1['fecha'] == pd.to_datetime('2017-07-01'), 'fechaHoraInicial']
#eventosRed1.loc[269, 'tramoLinea']
# listaTramos = \
#     ['Bengala_003\nBengala_008,JAGU_005, JAGU_009, JAGU_034, JAGU_36H, JAGU_043H, JGSW_004H, JGSW_008H\n']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_EMSA_Rural'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True

#### 4.6. Corrije nombre de tramos que empiezan con letras de la C a la J

In [111]:
#*******************************************************************************
# 4.6.1. Ajusta a '115 kV Puerto López - Campo Bonito' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
#eventosRed1.loc[eventosRed1['fecha'] == pd.to_datetime('2022-05-21'), ['tramoLinea', 'tramoAjus']]
# listaTramos = [' Disparo del circuito Puerto Lopez - Campo Bonito 115 KV.']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '115 kV Puerto López - Campo Bonito'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.6.2. Ajusta a '115 kV Reforma - Barzal' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Disparo del circuito Reforma - Barzal  115 kV.']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '115 kV Reforma - Barzal'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.6.3. Ajusta a '34_5kV_CCS' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Cepcolsa 1 34,5 KV']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.6.4. Ajusta a '34_5kV_CCS_CG' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Centro de Generación CCS']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_CG'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.6.5. Ajusta a '34_5kV_CCS_R2' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Falla en circuito de CCS E']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R2'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.6.6. Ajusta a '34_5kV_CCS_R3' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Falla en circuito de Campamento']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R3'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.6.7. Ajusta a '34_5kV_JGR_CG' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Centro de Generación JGR', 'Centro de Generación JGR y CCS', \
#     'Centro de Generacion Jaguar', 'Centro de Generación JGR']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JGR_CG'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.6.8. Ajusta a '34_5kV_JGR_R1' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['CDC-JGR Ramal 1-inyeccion 80K']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JGR_R1'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.6.9. Ajusta a '34_5kV_JSW' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['JGSW_004H, JGSW_008H']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JSW'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True

#### 4.7. Corrije nombre de tramos que empiezan con letras de la L a la R

In [145]:
#*******************************************************************************
# 4.7.1. Ajusta a '115 kV Puerto Gaitán - La Cristalina' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = [' línea La Cristalina 115 kV.', \
#     'La reparación quedo incluida en el contrato de GE, pero están pendiente la inclusión de los repuestos, se enviaron a catalogar']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '115 kV Puerto Gaitán - La Cristalina'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.2. Ajusta a '34_5kV_CCS' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Línea de entrada RC4, escalando la falla hasta el interruptor del circuito CEPCOLSA 1  en la subestación Gaitán 115', \
#     'Línea de entrada RC4, escalando la falla hasta el interruptor del circuito CEPCOLSA 1  en la subestación Gaitán 115', \
#         'Pozos  Caracara sur 34,5 kV', 'Ramal CCS', 'Ramal CCS CDC2', \
#             'Ramales CCS 34.5 KV', 'RC 04 CDC CCS 34.5 KV']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.3. Ajusta a '34_5kV_CCS_R1' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Ramal 1 Caracara sur 34,5 kV']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R1'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.4. Ajusta a '34_5kV_CCS_R1_2' y chequea 'tramoAjus'.Ok.
#*******************************************************************************
# listaTramos = ['RC Ramal clúster CCS C1-5']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R1_2'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.5. Ajusta a '34_5kV_CCS_R1_3' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Reconectador campamento CCS']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R1_3'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.6. Ajusta a '34_5kV_CCS_R2' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Ramal 2 Caracara sur 34,5 kV', 'Ramal 2 CCS MCDC-3-RC28', \
#     'Ramal 2 CDC_CCS', 'Ramal 2 CDC-CCS']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R2'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.7. Ajusta a '34_5kV_CCS_R2_1' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['RC MCDC 2-1 Ramal CCS A; MCDC 2-2']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R2_1'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.8. Ajusta a '34_5kV_CCS_R3' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Ramal 3 34,5 KV', 'Ramal 3 CCS 34.5 kV', \
#     'RC 1 MCDC 2-1-1 Ramal BGL 2', 'RC Ramal clúster BGL 3-8 y BGL 9']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R3'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.9. Ajusta a '34_5kV_JGR' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['RC 04 CDC JGR 34,5 KV']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JGR'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.10. Ajusta a '34_5kV_JGR_R1' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Ramal 1 CDC_JGR', 'Ramal 1 Jaguar 34,5 kV', \
#     'Ramal 1-1-5  34,5 KV Jaguar', 'RC 03 CDC JAGUAR RAMAL 1', \
#         'RC 20 Ramal jaguar 1 MCDC 3 - 34,5 KV Jaguar', 'RC 3 Ramal 1 JGR']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JGR_R1'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.11. Ajusta a '34_5kV_JGR_R1' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Ramal Jaguar 1-5']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JGR_R1'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.12. Ajusta a '34_5kV_JGR_R2' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Ramal 2 bengala  RC 2', 'Ramal 2 Jaguar', \
#     'Ramal 2 Jaguar 34.5 kV', 'Ramal 2 JGR']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JGR_R2'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.13. Ajusta a '34_5kV_JGR_R3' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Ramal 3 34,5 KV Jaguar', 'Ramal 3 Jaguar 34.5 kV']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JGR_R3'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.14. Ajusta a '34_5kV_JGR_R3' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['RC MCDC 3-1 Ramal JGR T7 y MCDC 3-2']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JGR_R3'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.15. Ajusta a '34_5kV_JSW' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['JGSW_004H, JGSW_008H', 'Ramal JSW', 'Ramal JSW 34,5 KV', \
#     'Ramal JSW CDC 3', 'Ramal JSW CDC2', 'Ramal JSW-CDC-3, JSW -5-7-8', \
#         'RC CDC 3 JSW 34,5 kV JSW 1-3']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JSW'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.7.16. Ajusta a '34_5kV_JSW' y chequea 'tramoAjus'.
#*******************************************************************************
listaTramos = ['JGSW_004H, JGSW_008H', 'Ramal JSW', 'Ramal JSW 34,5 KV', \
    'Ramal JSW CDC 3', 'Ramal JSW CDC2', 'Ramal JSW-CDC-3, JSW -5-7-8', \
        'RC CDC 3 JSW 34,5 kV JSW 1-3']
eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
    = '34_5kV_JSW'
eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
    = True

#### 4.8. Corrije nombre de tramos que empiezan con las letras RC

In [164]:
#*******************************************************************************
# 4.8.1. Ajusta a '34_5kV_CCS' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['RC 04 CDC CCS 34,5 KV', 'RC 04 CDC CCS 34.5 KV']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.8.2. Ajusta a '34_5kV_CCS_R1' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['RC Ramal clúster CCS C1-5']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R1'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.8.3. Ajusta a '34_5kV_CCS_R2' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
#eventosRed1.loc[eventosRed1['fecha'] == pd.to_datetime('2019-08-02'), ['tramoLinea', 'horaEvento', 'tramoAjus']]
# listaTramos = ['RC MCDC 2-1 Ramal CCS A; MCDC 2-2']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R2'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.8.4. Ajusta a '34_5kV_JGR' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['RC 04 CDC JGR 34,5 KV']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JGR'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.8.5. Ajusta a '34_5kV_JGR_R1' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['RC 03 CDC JAGUAR RAMAL 1', \
#     'RC 20 Ramal jaguar 1 MCDC 3 - 34,5 KV Jaguar', 'RC 3 Ramal 1 JGR']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JGR_R1'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.8.6. Ajusta a '34_5kV_JGR_R3' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['RC MCDC 3-1 Ramal JGR T7 y MCDC 3-2']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JGR_R3'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.8.7. Ajusta a '34_5kV_JSW' y chequea 'tramoAjus'.
#*******************************************************************************
listaTramos = ['RC CDC 3 JSW 34,5 kV JSW 1-3']
eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
    = '34_5kV_JSW'
eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
    = True

#### 4.9. Corrije nombre de tramos que empiezan con las letras S y la palabra troncal

In [178]:
#*******************************************************************************
# 4.9.1. Ajusta a '115 kV_SE_PG' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['TP1 CEPSA S/E Gaitán 115 KV']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '115 kV_SE_PG'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.9.2. Ajusta a '34_5kV_CCS' y chequea 'tramoAjus'. OK.
#*******************************************************************************
# listaTramos = ['Troncal Caracara sur 34,5 kV', 'Troncal CCS', \
#     'Troncal CCS - JSW 34,5 KV', 'Troncal CCS 34,5KV desde CDC-1; CDC_CCS']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.9.3. Ajusta a '34_5kV_CCS_R1' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['S/E CCS A10-14-18 34,5 kV', 'Subramal CCS 34,5kV']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R1'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.9.4. Ajusta a '34_5kV_CCS_R3' y chequea 'tramoAjus'.
#*******************************************************************************
# listaTramos = ['Reconectador campamento CCS']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R3'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.9.5. Ajusta a '34_5kV_JGR' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Troncal 34,5 KV Jaguar', 'Troncal Jaguar  34,5 kV', \
#     'Troncal JGR 34,5 kV', 'Troncal JGR 34,5KV desde CDC-1; CDC_CCS']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JGR'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.9.6. Ajusta a '34_5kV_JSW' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Troncal JSW 34,5 kV']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JSW'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.9.7. Ajusta a '34_5kV_SE_PG' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Troncales Caracara, Jaguar y JSW']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_SE_PG'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True

#### 4.10. Corrije el nombre de los 50 tramos que están pendientes por corregir

In [19]:
#pd.unique(eventosRed1.loc[eventosRed1['tramoAjus'] == False, 'tramoLinea'])
#*******************************************************************************
# 4.10.1. Ajusta a '34_5kV_CCS' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5KV Ramales 34,5KV CCS', '34,5 CDC CCS 34,5 KV', \
#     '34,5KV  Troncal Caracara sur.', '34,5 kV RC 04 CDC CCS', \
#         '34,5 kV Troncal CCS - JSW']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.10.2. Ajusta a '34_5kV_CCS_CG' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 Troncal CCS']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_CG'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.10.3. Ajusta a '34_5kV_CCS_R2' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34.5 kV SDT BENGALA 2']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_CCS_R2'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.10.4. Ajusta a '34_5kV_JGR' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Troncal Jaguar 34,5 kV', '34,5 Troncal Jaguar', \
#     '34,5 kV RC 04 CDC JGR;', '34,5 kV Centro generación Jaguar']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JGR'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.10.5. Ajusta a '34_5kV_JSW' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['Troncal JSW 34,5 kV ', '34.5 kV Ramal JSW', \
#     '34,5 CDC JSW 34,5 KV']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_JSW'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True
#*******************************************************************************
# 4.10.6. Ajusta a '34_5kV_SE_PG' y chequea 'tramoAjus'. Ok.
#*******************************************************************************
# listaTramos = ['34,5 kV RC 04 CDC JGR; 34,5 kV RC 04 CDC CCS']
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos),'tramoLinea'] \
#     = '34_5kV_SE_PG'
# eventosRed1.loc[eventosRed1['cadenaTramoLinea'].isin(listaTramos), 'tramoAjus'] \
#     = True

In [24]:
#eventosRed1.loc[eventosRed1['tramoAjus'] == True, 'tramoAjus'].count()
#eventosRed1.loc[eventosRed1['tramoAjus'] == True, ['tramoLinea', 'tramoAjus']]
#eventosRed1.loc[eventosRed1['tramoAjus'] == False, ['tramoLinea', 'tramoAjus']]
#eventosRed1.info()
pd.unique(eventosRed1.loc[:, 'tramoLinea'])

array(['115 kV Campo Bonito - Puerto Gaitán',
       '115 kV Puerto López - Campo Bonito', '34_5kV_JSW', '34_5kV_SE_PG',
       '34_5kV_CCS', '34_5kV_CCS_CG', '115 kV San Fernando', '34_5kV_JGR',
       '115 kV Puerto Gaitán - La Cristalina', '34_5kV_CCS_R1',
       '34_5kV_JGR_R2', '115 kV Reforma - Barzal', '34_5kV_JGR_CG',
       '115 kV Suria - Puerto López', '115 kV Ocoa - Suria',
       '34_5kV_EMSA_Rural', '115 kV Reforma - Ocoa', '34_5kV_JGR_R1',
       '115 kV_SE_PG', '34_5kV_JGR_R3', '230 kV Reforma', '34_5kV_CCS_R3',
       '34_5kV_CCS_R2', '115 kV Subestación Puerto Gaitán',
       '115 kV Ocoa - Santa Helena'], dtype=object)

In [21]:
#Salva eventosRed1 en eventosRed2, con formato parquet (más comprimido).
#eventosRed1.to_parquet("parquet/eventosRed2.parquet.gzip", compression='gzip')
#Lee el archivo eventosRed en formato parquet.
#eventosRed1 = pd.read_parquet("parquet/eventosRed2.parquet.gzip")  